In [ ]:
import findspark

In [ ]:
findspark.init("/Users/sharmilisrinivasan/spark-2.4.3-bin-hadoop2.7/")

In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession \
    .builder \
    .master("spark://localhost:7077") \
    .appName("Spark_ES_exp") \
    .config("spark.jars", "elasticsearch-spark-20_2.11-7.3.0.jar") \
    .getOrCreate()

In [ ]:
spark

In [ ]:
from spark_es import SparkESConnector

In [ ]:
spark_es_connector = SparkESConnector(spark, "localhost:9200")

## Read

In [ ]:
index_to_read = "sample-detections"
read_query = """
{
  "query": {
    "term": {
      "is_anomaly": {
        "value": 1
      }
    }
  }
}
"""
fields_to_read = ["@timestamp","log_template","is_anomaly"]
rename_fields = ["Timestamp", "Template","Anomalous"]

In [ ]:
data_read = spark_es_connector.read_index(index_to_read, read_query, fields_to_read, rename_fields)

In [ ]:
data_read.show()

## Write

In [ ]:
index_to_write = "es_spark_test_index"
df_to_wrtie = data_read
rename_columns = ["@timestamp", "log_template","anomaly_flag"]

In [ ]:
spark_es_connector.write_index(index_to_write, df_to_wrtie, renames_=rename_columns)